In [16]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [6]:
df2 = pd.read_csv('C:/Users/DMQA/DATA4.csv', encoding = 'euc-kr')

In [9]:
new_df2 = df2.copy()
list_df2 = np.array(df2['review'].tolist())

In [10]:
import re

def cleansing(text):
    cl_text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…/ ]', ' ', text)
    return cl_text

c_list_df2 = []
for x in list_df2 : 
     c_list_df2.append(cleansing(x))

new_df2['review'] = new_df2['review'].apply(cleansing)
new_df2[:10]

,review,ord,ord_group
0,후미권 이끌었고 직선 순위도약 시도나 추입력 발휘안돼 고전,7,0
1,선행으로 경주를 주도했으나 직선막판 근소한 차이로 덜미잡혀,2,1
2,경주전반에 걸쳐 후미권을 벗어나지 못하고 따라가는모습,10,0
3,선두권을 압박하며 따라갔으나 4코너부터 격차벌어지며 고전함,6,0
4,경주전반에 걸쳐 후미권을 벗어나지 못하고 따라가는모습,8,0
5,선행마 후미를 따라가며 기회노렸으나 직선 근소한 차이로 덜미,4,0
6,경주전반에 걸쳐 후미권을 벗어나지 못하고 따라가는모습,11,0
7,초반 선입권 가담이나 2코너이후 순위밀려 직선 후미권으로,10,0
8,경주전반에 걸쳐 후미권을 벗어나지 못하고 따라가는모습,10,0
9,주행심사 경주능력부진 1위마와 30 0 마신차,12,0


In [13]:
from hanspell import spell_checker

def handspell_test(text) :
    spelled_text = spell_checker.check(text)
    handspell_text = spelled_text = spelled_text.checked
    return handspell_text

In [18]:
new_df2['review'] = new_df2['review'].apply(handspell_test)

In [19]:
new_df2

,review,ord,ord_group
0,후미권 이끌었고 직선 순위 도약 시도나 추입력 발휘 안돼 고전,7,0
1,선행으로 경주를 주도했으나 직선 막판 근소한 차이로 덜미잡혀,2,1
2,경주 전반에 걸쳐 후미권을 벗어나지 못하고 따라가는 모습,10,0
3,선두권을 압박하며 따라갔으나 4코너부터 격차 벌어지며 고전함,6,0
4,경주 전반에 걸쳐 후미권을 벗어나지 못하고 따라가는 모습,8,0
...,...,...,...
5227,게이트 극복하며 선행 탄력 유지 우승,1,1
5228,시종 후미 직선 탄력 큰 변화 아님,9,0
5229,선두권 후 미 공략이나 직선 강한 모습을 못 보여줌,6,0
5230,스포츠조선배 후미권에서 순위 좁히지 못해,5,0


In [53]:
train = new_df2.sample(frac=0.8, random_state=42)
len(trainset)

4186

In [54]:
test = new_df2.drop(train.index)
len(testset)

1046

In [55]:
import logging
import os
import unicodedata
from shutil import copyfile
 
from transformers import PreTrainedTokenizer
 
 
logger = logging.getLogger(__name__)
 
VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}
 
PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}
 
PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}
 
PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}
 
SPIECE_UNDERLINE = u'▁'
 
 
class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES
 
    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )
 
        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)
 
        self.model_max_length_single_sentence = self.model_max_length - 2  # take into account special tokens
        self.model_max_length_sentences_pair = self.model_max_length - 3  # take into account special tokens
 
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
 
        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt
 
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)
 
    @property
    def vocab_size(self):
        return len(self.idx2token)
 
    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state
 
    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)
 
    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')
 
        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()
 
        return outputs
 
    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)
 
        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)
 
        return new_pieces
 
    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])
 
    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]
 
    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string
 
    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A RoBERTa sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep
 
    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """
 
        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))
 
        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]
 
    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]
 
    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return
 
        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])
 
        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)
 
        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1
 
        return out_vocab_model, out_vocab_txt


In [56]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

In [57]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['▁보는', '내', '내', '▁그대로', '▁들어', '맞', '는', '▁예측', '▁카리스마', '▁없는', '▁악', '역']


In [58]:
train[100:110]

,review,ord,ord_group
2847,선행 강공 나섰고 직선 격차 벌이며 우승,1,1
4766,양호한 출발로 선두권 압박 전개나 직선 격차 못 좁혀,3,0
4223,출발 밀려 후미권 이끌었고 직선 올라오나 역전 실패,3,0
4490,중반 이후 밀리는 모습에 직선에서도 걸음 무뎌짐,9,0
4820,시종 후미권 전개나 직선 걸음을 보여줌,7,0
4543,시종 중간 후미에 직선에서도 덜미잡혀 밀려남,9,0
239,중간 후 미 머물렀으나 직선 추입 나서며 4착,4,0
4112,중위권 최선 전개나 직선 걸음 무뎌지며 밀려남,8,0
107,중위권 전개하다 직선 추입력을 보여주며 막판 후 착 각축,3,0
3999,선두권 압박하다 직선 선행마 호미나 역전 실패,2,1


In [59]:
corpus_train = np.array(train['review'].tolist())
print(corpus_train[100:110])

['선행 강공 나섰고 직선 격차 벌이며 우승' '양호한 출발로 선두권 압박 전개나 직선 격차 못 좁혀'
 '출발 밀려 후미권 이끌었고 직선 올라오나 역전 실패' '중반 이후 밀리는 모습에 직선에서도 걸음 무뎌짐'
 '시종 후미권 전개나 직선 걸음을 보여줌' '시종 중간 후미에 직선에서도 덜미잡혀 밀려남'
 '중간 후 미 머물렀으나 직선 추입 나서며 4착' '중위권 최선 전개나 직선 걸음 무뎌지며 밀려남'
 '중위권 전개하다 직선 추입력을 보여주며 막판 후 착 각축' '선두권 압박하다 직선 선행마 호미나 역전 실패']


In [63]:
for x in corpus_train[100:110]:
    print(tokenizer.tokenize(x))

['▁선', '행', '▁강', '공', '▁나섰', '고', '▁직', '선', '▁격차', '▁벌', '이며', '▁우승']
['▁양', '호', '한', '▁출발', '로', '▁선두', '권', '▁압박', '▁전개', '나', '▁직', '선', '▁격차', '▁못', '▁', '좁', '혀']
['▁출발', '▁밀려', '▁후', '미', '권', '▁이끌', '었고', '▁직', '선', '▁올라', '오', '나', '▁역전', '▁실패']
['▁중반', '▁이후', '▁밀', '리는', '▁모습', '에', '▁직', '선', '에서도', '▁', '걸음', '▁무', '뎌', '짐']
['▁시', '종', '▁후', '미', '권', '▁전개', '나', '▁직', '선', '▁', '걸음', '을', '▁보여', '줌']
['▁시', '종', '▁중간', '▁후', '미', '에', '▁직', '선', '에서도', '▁덜', '미', '잡', '혀', '▁밀려', '남']
['▁중간', '▁후', '▁미', '▁머물', '렀', '으나', '▁직', '선', '▁추', '입', '▁나서', '며', '▁4', '착']
['▁중', '위권', '▁최', '선', '▁전개', '나', '▁직', '선', '▁', '걸음', '▁무', '뎌', '지', '며', '▁밀려', '남']
['▁중', '위권', '▁전개', '하다', '▁직', '선', '▁추', '입', '력을', '▁보여주', '며', '▁막판', '▁후', '▁착', '▁각', '축']
['▁선두', '권', '▁압박', '하다', '▁직', '선', '▁선', '행', '마', '▁호', '미', '나', '▁역전', '▁실패']
